# Swim TCSP Import CSVs

In [4]:
# pip install pymysql

Note: you may need to restart the kernel to use updated packages.


## Connection and Test

In [3]:
import csv
import pymysql

def connect_to_tcsp():
    connection_details = {
        "host": 'tcsp.ie',
        "port": 3306,
        "user": 't567715',
        "password": '0bjs8Pz55Q',
        "database": 't567715_wp_tcsp',
        "charset": 'utf8mb4',
        "cursorclass": pymysql.cursors.DictCursor
    }
    try:
        connection = pymysql.connect(**connection_details)
        return connection
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL Database: {e}")
        return None



In [4]:
def fetch_mor_events(connection):
    if connection is None:
        return []
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT id, event FROM mor_events")
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

def save_data_to_csv(data, filename):
    if data:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)

# Main execution
try:
    conx = connect_to_tcsp()
    if conx is not None:
        data = fetch_mor_events(conx)
        save_data_to_csv(data, 'public_swim_catagories.csv')
finally:
    if conx:
        conx.close()


## Multiple Files

In [6]:
import csv
import pymysql

def fetch_mor_events(connection):
    if connection is None:
        return []
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT id, event FROM mor_events")
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

def fetch_terms_table(connection):
    if connection is None:
        return []
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT start_date, finish_date,  COALESCE(rebook_start, '2000-01-01') AS rebook_start,  "
                           "COALESCE(booking_switch_date, '2000-01-01') AS booking_switch_date, COALESCE"
                           "(assesments_complete, '2000-01-01') AS assesments_complete FROM mor_terms")
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

def save_data_to_csv(data, filename):
    if data:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)

# Main execution
try:
    conx = connect_to_tcsp()  # Assuming this establishes your database connection
    if conx is not None:
        mor_events_data = fetch_mor_events(conx)
        terms_data = fetch_terms_table(conx)

        save_data_to_csv(mor_events_data, 'public_swim_categories.csv')
        save_data_to_csv(terms_data, 'terms.csv')
finally:
    if conx:
        conx.close()
